In [ ]:
import pandas as pd
import numpy as np
from yaml import safe_load
import datetime

import yfinance as yf

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
with open("portfolio.yaml", "rb") as f:
    portfolio = safe_load(f)

In [ ]:
datetime.datetime.today()

In [ ]:
# Transform dictionary into a list of records
records = []
for ticker, transactions in portfolio.items():
    for date, details in transactions.items():
        qte = next((item['QTE'] for item in details if 'QTE' in item), None)
        buy_price = next((pd.to_numeric(item['PRICE'], errors="coerce") for item in details if 'PRICE' in item), None)
        buy_price_yahoo = yf.download(ticker, start=date, end=date + datetime.timedelta(1), group_by="ticker")[ticker]["Close"]
        today_price = yf.download(
            ticker, start=datetime.datetime.today(), end=datetime.datetime.today() + datetime.timedelta(1), 
            group_by="ticker"
        )[ticker]["Close"]

        if not buy_price_yahoo.empty:
            records.append({
                'date': date, 'ticker': ticker, 'quantity': qte, 'buy_price': buy_price, 'buy_price_yahoo': buy_price_yahoo.values[0],
                'today_price': today_price.values[0]
            })
        else:
            records.append({
                'date': date, 'ticker': ticker, 'quantity': qte, 'buy_price': buy_price, 'buy_price_yahoo': np.nan,
                'today_price': today_price.values[0]
            })


# Convert to DataFrame
df = pd.DataFrame(records)

# Sort by Date
df.sort_values(by=["ticker", "date"], inplace=True)

# Reset index
df.reset_index(drop=True, inplace=True)
df.loc[df.buy_price.isna(), "buy_price"] = df.loc[df.buy_price.isna(), "buy_price_yahoo"]

df.drop(columns=["buy_price_yahoo"], inplace=True)


In [ ]:
hist_data = yf.download(
    list(portfolio.keys()), start="2021-01-01", 
    group_by="ticker"
)
hist_data = hist_data.stack(level=0, future_stack=True)
hist_data.columns = hist_data.columns.values
hist_data = hist_data.reset_index()
hist_data = hist_data[["Date", "Ticker", "Close"]]

In [ ]:
df["cumulative_quantity"] = df.groupby("ticker").quantity.cumsum()

In [ ]:
# Set start and end date
start_date = pd.Timestamp("2020-02-26")
end_date = df["date"].max()

# Create a complete date range
all_dates = pd.date_range(start=start_date, end=end_date, freq="D")

# Create a MultiIndex with all tickers and all dates
multi_index = pd.MultiIndex.from_product([df["ticker"].unique(), all_dates], names=["ticker", "date"])

# Reindex the DataFrame to expand the dates for all tickers
full_df = df[["ticker", "date", "buy_price", "cumulative_quantity"]].set_index(["ticker", "date"]).reindex(multi_index)

# Forward-fill the Cumulative_Quantity column
full_df["cumulative_quantity"] = full_df.groupby("ticker")["cumulative_quantity"].ffill()
full_df["cumulative_quantity"] = full_df["cumulative_quantity"].fillna(0)
# full_df["buy_price"] = full_df["buy_price"].ffill()

# Reset index for final format
full_df = full_df.reset_index()

full_df

In [ ]:
df

In [ ]:
sns.lineplot(x="date", y="cumulative_quantity", hue="ticker", data=full_df)

In [ ]:
df["spendings"] = df["buy_price"] * df["quantity"]

In [ ]:
(
    df.groupby("ticker").quantity.sum()
    * df.groupby("ticker").today_price.first()
) - df.groupby("ticker").balance.sum()

In [ ]:
(
    df.groupby("ticker").quantity.sum()
    * df.groupby("ticker").today_price.first()
).sum() - df.groupby("ticker").balance.sum().sum()

In [ ]:
(
    df.groupby("ticker").quantity.sum()
    * df.groupby("ticker").buy_price.first()
).sum()

In [ ]:
df.groupby("ticker").balance.first()